In [ ]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [ ]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [ ]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

In [5]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

iteration_list = [5, 6]
for iteration in iteration_list:
    for history_size in history_list:
        for future_size in future_list:
            print(f"{iteration}th iteration")
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
    print('\n\n')
print('end opitmization')

5th iteration
history size: 10
future size: 10
model is loaded from: ./model/10_10_1_50_1_50_datt_seq2seq_gru_5


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.95191,0.940408,0.968614,0.953644,3.560584,4.268898,2.762227,3.53057
1,0.890604,0.893705,0.927472,0.903927,5.335878,5.702014,4.197904,5.078599
2,0.803316,0.846414,0.888765,0.846165,7.057305,6.854641,5.197486,6.36981
3,0.705844,0.807029,0.856355,0.789743,8.516202,7.684496,5.904804,7.3685
4,0.606356,0.781837,0.834288,0.740827,9.755366,8.171844,6.340142,8.089117
5,0.508723,0.762108,0.816329,0.69572,10.793311,8.534796,6.673193,8.6671
6,0.416878,0.743919,0.802251,0.654349,11.6476,8.855826,6.922954,9.142127
7,0.33084,0.726027,0.793323,0.61673,12.447698,9.160305,7.076812,9.561605
8,0.255451,0.709155,0.786049,0.583552,13.145387,9.438589,7.199726,9.927901




5th iteration
history size: 10
future size: 20
model is loaded from: ./model/10_20_1_50_1_50_datt_seq2seq_gru_5


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.907338,0.9236,0.942273,0.924404,6.0125,4.829025,3.738263,4.859929
1,0.831534,0.876888,0.906706,0.871709,7.959419,6.131919,4.75183,6.281056
2,0.749568,0.83968,0.867606,0.818951,9.280731,7.000062,5.659884,7.313559
3,0.664425,0.810446,0.822773,0.765882,10.384433,7.614201,6.546914,8.18185
4,0.576804,0.792378,0.778699,0.71596,11.475779,7.972397,7.313586,8.920587
5,0.489491,0.776837,0.750726,0.672352,12.562948,8.269437,7.759246,9.530544
6,0.411084,0.760598,0.734248,0.63531,13.229992,8.567639,8.009402,9.935678
7,0.344047,0.741391,0.717437,0.600958,13.69563,8.907545,8.256984,10.286719
8,0.286228,0.719188,0.696239,0.567218,14.074536,9.285111,8.559255,10.639634




5th iteration
history size: 10
future size: 30
model is loaded from: ./model/10_30_1_50_1_50_datt_seq2seq_gru_5


...

csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_6.csv


6th iteration
history size: 40
future size: 30
Epoch 1/10000
559/559 - 8s - loss: 0.3757 - val_loss: 0.3147 - 8s/epoch - 15ms/step
Epoch 2/10000
559/559 - 5s - loss: 0.2873 - val_loss: 0.2887 - 5s/epoch - 8ms/step
Epoch 3/10000
559/559 - 5s - loss: 0.2674 - val_loss: 0.2747 - 5s/epoch - 8ms/step
Epoch 4/10000
559/559 - 5s - loss: 0.2517 - val_loss: 0.2639 - 5s/epoch - 8ms/step
Epoch 5/10000
559/559 - 5s - loss: 0.2394 - val_loss: 0.2457 - 5s/epoch - 8ms/step
Epoch 6/10000
559/559 - 5s - loss: 0.2271 - val_loss: 0.2349 - 5s/epoch - 8ms/step
Epoch 7/10000
559/559 - 5s - loss: 0.2132 - val_loss: 0.2294 - 5s/epoch - 8ms/step
Epoch 8/10000
559/559 - 5s - loss: 0.2040 - val_loss: 0.2099 - 5s/epoch - 8ms/step
Epoch 9/10000
559/559 - 5s - loss: 0.1930 - val_loss: 0.2014 - 5s/epoch - 8ms/step
Epoch 10/10000
559/559 - 5s - loss: 0.1854 - val_loss: 0.1923 - 5s/epoch - 8ms/step
Epoch 11/10000
559/559 - 5s - loss: 0.1779 - val_

INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_6\assets


INFO:tensorflow:Assets written to: ./model/40_30_1_50_1_50_datt_seq2seq_gru_6\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.852236,0.868947,0.916844,0.879342,7.583139,6.365358,4.417085,6.121861
1,0.768344,0.822075,0.872841,0.821086,9.496882,7.417326,5.464852,7.459687
2,0.671438,0.770902,0.826208,0.756183,11.312529,8.417414,6.391821,8.707255
3,0.567192,0.730297,0.779179,0.692223,12.985877,9.134309,7.207735,9.775974
4,0.462332,0.700652,0.73121,0.631398,14.474273,9.625151,7.95489,10.684771
5,0.361785,0.679041,0.682022,0.574283,15.768867,9.969036,8.654352,11.464085
6,0.272064,0.66079,0.633353,0.522069,16.839339,10.251006,9.295261,12.128535
7,0.197545,0.643655,0.588465,0.476555,17.678873,10.50907,9.850338,12.679427
8,0.137116,0.624955,0.544748,0.435606,18.331332,10.784162,10.362683,13.159392


csv file is saved to: ./result/40_30_1_50_1_50_datt_seq2seq_gru_6.csv





end opitmization
